# Demo Tools - Fixed Version

## Imports

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import (
    HumanMessage, 
    SystemMessage, 
    ToolMessage
)
from langchain.tools import tool
from dotenv import load_dotenv
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import beta

In [ ]:
load_dotenv()

## Initialize LLM

In [ ]:
llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0.0,
)

## Tool Creation

In [ ]:
@tool
def multiply(a: int, b: int) -> int:
    """Multiply two numbers."""
    return a * b

In [ ]:
@tool
def plot_beta_distribution(alpha: float, beta_param: float, n_points: int = 1000) -> str:
    """
    Plots the Beta distribution for given alpha and beta parameters.
    Use this when asked to plot, visualize, or show a beta distribution.
    
    Args:
        alpha (float): Shape parameter α (> 0)
        beta_param (float): Shape parameter β (> 0)
        n_points (int): Number of points in the x-axis grid
    
    Returns:
        str: Confirmation message
    """
    # Create x values between 0 and 1
    x = np.linspace(0, 1, n_points)
    
    # Compute the Beta PDF for each x
    y = beta.pdf(x, alpha, beta_param)
    
    # Plot
    plt.figure(figsize=(8, 4))
    plt.plot(x, y, 'b-', lw=2, label=f'Beta(α={alpha}, β={beta_param})')
    plt.title("Beta Distribution", fontsize=14)
    plt.xlabel("x", fontsize=12)
    plt.ylabel("Density", fontsize=12)
    plt.legend()
    plt.grid(alpha=0.3)
    plt.show()
    
    return f"Successfully plotted Beta distribution with α={alpha}, β={beta_param}"

## Setup Tools

In [ ]:
tools = [multiply, plot_beta_distribution]
tool_map = {tool.name: tool for tool in tools}
print(f"Available tools: {list(tool_map.keys())}")

## Bind Tools to LLM

In [ ]:
# Use tool_choice="any" to force the LLM to always use tools
llm_with_tools = llm.bind_tools(tools, tool_choice="any")

## Example 1: Multiply Tool

In [ ]:
# RESET: Create fresh messages for this example
question = "What is 3 multiplied by 2?"
messages = [
    SystemMessage("You're a helpful assistant. Use tools when available."),
    HumanMessage(question)
]

print(f"Question: {question}")
print()

In [ ]:
# Step 1: LLM decides to call a tool
ai_message = llm_with_tools.invoke(messages)

print("LLM Response:")
print(f"Content: '{ai_message.content}'")
print(f"Tool calls: {ai_message.tool_calls}")
print()

# Add to message history
messages.append(ai_message)

In [ ]:
# Step 2: Execute the tool(s)
if ai_message.tool_calls:
    for tool_call in ai_message.tool_calls:
        tool_call_id = tool_call['id']
        function_name = tool_call['name']
        arguments = tool_call['args']
        
        print(f"Executing: {function_name}({arguments})")
        
        # Execute the tool
        func = tool_map[function_name]
        result = func.invoke(arguments)
        
        print(f"Result: {result}")
        print()
        
        # Create tool message with result
        tool_message = ToolMessage(
            content=str(result),
            name=function_name,
            tool_call_id=tool_call_id,
        )
        messages.append(tool_message)
else:
    print("No tool calls made.")

In [ ]:
# Step 3: Get final formatted response from LLM
final_response = llm_with_tools.invoke(messages)

print("Final Answer:")
print(final_response.content)

## Example 2: Plot Beta Distribution Tool

In [ ]:
# RESET: Create fresh messages for this example
question = "Plot a beta distribution with alpha=2 and beta=5"
messages = [
    SystemMessage("You're a helpful assistant. Use tools when available."),
    HumanMessage(question)
]

print(f"Question: {question}")
print()

In [ ]:
# Step 1: LLM decides to call a tool
ai_message = llm_with_tools.invoke(messages)

print("LLM Response:")
print(f"Content: '{ai_message.content}'")
print(f"Tool calls: {ai_message.tool_calls}")
print()

# Add to message history
messages.append(ai_message)

In [ ]:
# Step 2: Execute the tool(s)
if ai_message.tool_calls:
    for tool_call in ai_message.tool_calls:
        tool_call_id = tool_call['id']
        function_name = tool_call['name']
        arguments = tool_call['args']
        
        print(f"Executing: {function_name}({arguments})")
        print()
        
        # Execute the tool (this will display the plot)
        func = tool_map[function_name]
        result = func.invoke(arguments)
        
        print(f"Result: {result}")
        print()
        
        # Create tool message with result
        tool_message = ToolMessage(
            content=str(result),
            name=function_name,
            tool_call_id=tool_call_id,
        )
        messages.append(tool_message)
else:
    print("No tool calls made.")

In [ ]:
# Step 3: Get final formatted response from LLM
final_response = llm_with_tools.invoke(messages)

print("Final Answer:")
print(final_response.content)

## Helper Function: Complete Tool Calling Flow

In [ ]:
def ask_with_tools(question: str, llm_with_tools, tool_map, verbose=True):
    """
    Complete tool calling flow:
    1. LLM decides which tool to call
    2. Execute the tool
    3. LLM formats final answer
    """
    messages = [
        SystemMessage("You're a helpful assistant. Use tools when available."),
        HumanMessage(question)
    ]
    
    if verbose:
        print(f"Question: {question}")
        print("=" * 60)
    
    # Step 1: LLM decides to call tools
    ai_message = llm_with_tools.invoke(messages)
    messages.append(ai_message)
    
    if verbose and ai_message.tool_calls:
        print(f"\nLLM wants to call {len(ai_message.tool_calls)} tool(s)")
    
    # Step 2: Execute tools
    if ai_message.tool_calls:
        for tool_call in ai_message.tool_calls:
            function_name = tool_call['name']
            arguments = tool_call['args']
            
            if verbose:
                print(f"\nExecuting: {function_name}({arguments})")
            
            # Execute the tool
            func = tool_map[function_name]
            result = func.invoke(arguments)
            
            # Add result to messages
            tool_message = ToolMessage(
                content=str(result),
                name=function_name,
                tool_call_id=tool_call['id'],
            )
            messages.append(tool_message)
    
    # Step 3: Get final answer
    final_response = llm_with_tools.invoke(messages)
    
    if verbose:
        print("\n" + "=" * 60)
        print("Final Answer:")
        print(final_response.content)
    
    return final_response.content

## Test with Helper Function

In [ ]:
# Test multiplication
ask_with_tools(
    "What is 7 times 8?",
    llm_with_tools,
    tool_map
)

In [ ]:
# Test plotting
ask_with_tools(
    "Show me a beta distribution with alpha=5 and beta=2",
    llm_with_tools,
    tool_map
)

## Key Takeaways

1. **Tool calls are in `ai_message.tool_calls`** - Don't use `additional_kwargs`
2. **Use `tool_choice="any"`** to force tool usage
3. **Two LLM invocations needed**:
   - First: LLM decides which tool to call
   - Second: LLM formats the final answer
4. **Tools should return strings** for easy integration with `ToolMessage`
5. **Message flow**: SystemMessage → HumanMessage → AIMessage (tool call) → ToolMessage (result) → AIMessage (final answer)

## Troubleshooting

**Error: "An assistant message with 'tool_calls' must be followed by tool messages"**

This means you have a tool call in your message history that was never executed. 

**Cause:** Cells were run out of order.

**Fix:** Re-run the "RESET" cell (the first cell in each example) to create fresh messages, then run cells in order:
1. RESET cell (creates messages)
2. Step 1: LLM decides tool
3. Step 2: Execute tool
4. Step 3: Get final answer